In [77]:
import os
import json
import pandas as pd
import traceback


In [78]:
!pip install langchain-community

In [79]:
from langchain.chat_models import ChatOpenAI

    

In [112]:
from dotenv import load_dotenv

load_dotenv()


True

In [117]:
KEY = os.getenv("OPENAI_API_KEY")


In [118]:
llm = ChatOpenAI(openai_api_key = KEY,model_name="gpt-3.5-turbo",temperature=0.5)

In [83]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2
from langchain.chains import LLMChain


In [84]:
RESPONSE_JSON = {
    "1": {
        "mcq" : "multiple choice question",
        "options": {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",

    },
    "2": {
        "mcq" : "multiple choice question",
        "options": {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",

    },
    "3": {
        "mcq" : "multiple choice question",
        "options": {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",

    },
    "4": {
        "mcq" : "multiple choice question",
        "options": {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",

    },"5": {
        "mcq" : "multiple choice question",
        "options": {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",

    },
    "6": {
        "mcq" : "multiple choice question",
        "options": {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",

    },
    "7": {
        "mcq" : "multiple choice question",
        "options": {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",

    },
    "8": {
        "mcq" : "multiple choice question",
        "options": {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",

    },
    "9": {
        "mcq" : "multiple choice question",
        "options": {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",

    },
    "10": {
        "mcq" : "multiple choice question",
        "options": {
            "a" : "choice here",
            "b" : "choice here",
            "c" : "choice here",
            "d" : "choice here",
        },
        "correct" : "correct answer",

    },

}

In [85]:
TEMPLATE = """
Text :{text}
You are an expert MCQ maker. Given the above text , it is your job to \
create a Quiz of {number} multiple choice Questons for {subject} students in {tone} tone.
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like RESPONSE_JSON below and use it as a guide.\
insure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [86]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template = TEMPLATE
)

In [87]:
quiz_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt, output_key = "quiz", verbose = True)


In [88]:
TEMPLATE2 = """
You are and expert english grammerian and writer . Given a Multiple Choice for {subject} students.\
You read to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity
if the quiz is not at per the with the cognitive and analytical abilties of the students,\
update the quiz questions with needs to be changed and change the tone such that it perfectly fits the students.
Quiz_MCQs:
{quiz}

Check froma nd expert English writer of the above Quiz:
""" 

In [89]:
quiz_evaluation_prompt = PromptTemplate(input_variables=["subject", "quiz"], template = TEMPLATE)

In [90]:
review_chain = LLMChain(llm = llm, prompt = quiz_generation_prompt, output_key = "review", verbose = True)

In [91]:
generate_evaluate_chain = SequentialChain(chains = [quiz_chain, review_chain], input_variables = ["text", "number", "subject", "tone", "response_json"],
                                           output_variables = ["quiz", "review"], verbose = True,)


In [92]:
file_path =r"D:\new_for_project\mcqgen\data.txt"

In [93]:
file_path

'D:\\new_for_project\\mcqgen\\data.txt'

In [94]:
with open(file_path, 'r') as file:

    TEXT = file.read()

In [95]:
print(TEXT)

The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied human cognitive systems contributed 

In [96]:
#serialize the python dictionary into a json formatted string
json.dumps(RESPONSE_JSON)


'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "4": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "5": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "6": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "7": {"mcq": "

In [97]:
NUMBER = 4
SUBJECT = "machine learning"
TONE = "simple"


In [119]:
#https://python.langchain.com/docs/modules/model_io/llms/token_usage_tracking

#How to setup Token Usage Tracking in LangChain
with get_openai_callback() as cb:
    response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
    )

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

Text :The term machine learning was coined in 1959 by Arthur Samuel, an IBM employee and pioneer in the field of computer gaming and artificial intelligence.[8][9] The synonym self-teaching computers was also used in this time period.[10][11]

Although the earliest machine learning model was introduced in the 1950s when Arthur Samuel invented a program that calculated the winning chance in checkers for each side, the history of machine learning roots back to decades of human desire and effort to study human cognitive processes.[12] In 1949, Canadian psychologist Donald Hebb published the book The Organization of Behavior, in which he introduced a theoretical neural structure formed by certain interactions among nerve cells.[13] Hebb's model of neurons interacting with one another set a groundwork for how AIs and machine learning algorithms work under nodes, or artificial neurons used by computers to communicate data.[12] Other researchers who have studied huma

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: sk-proj-********************************************************************************************************************************************************hLkA. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [99]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:0
Prompt Tokens:0
Completion Tokens:0
Total Cost:0.0


In [100]:
response

NameError: name 'response' is not defined

In [101]:
response.get("quiz")

NameError: name 'response' is not defined

In [102]:
quiz = json.loads(quiz)

TypeError: the JSON object must be str, bytes or bytearray, not DataFrame

In [103]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [104]:
quiz_table_data

[]

In [105]:
quiz = pd.DataFrame(quiz_table_data)

In [106]:
quiz.to_csv("machinelearning.csv",index=False)

In [107]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'12_01_2024_10_17_35'